# Dataset Preparation for First ICU Prediction

## Imports & Inits

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import numpy as np
import pandas as pd
import spacy

from pathlib import Path

In [3]:
path = Path('./data')

## Dataset Processing

1. Load in the data
2. Drop duplicates
3. Merge `category`, `description`, and `text` into a new column called `note`
4. Tokenize text using `scispacy` and create new column called `scispacy_note` to save tokenized text
5. Save the relevant columns to a csv file onto disk

In [4]:
def tokenize_text(text):
  tokens = [token.text for token in nlp(text)]
  return ' '.join(tokens)

def group_eth(eth):
  eth = eth.lower()
  if 'white' in eth:
    return 'white'
  elif 'black' in eth:
    return 'black'
  elif 'hispanic' in eth:
    return 'hispanic'
  elif 'asian' in eth:
    return 'asian'
  else:
    return 'unknown'

In [5]:
nlp = spacy.load('en_core_sci_md', disable=['parser', 'ner', 'tagger'])

In [6]:
df = pd.read_csv(path/'raw_dataset.csv')
df.drop_duplicates(inplace=True)
df['note'] = df['category'].str.cat(df['description'], sep='\n')
df['note'] = df['note'].str.cat(df['text'], sep='\n')
df['ethnicity'] = df['ethnicity'].apply(group_eth)
df['scispacy_note'] = df['note'].apply(tokenize_text)
df.head()

,subject_id,hadm_id,icustay_id,admission_type,admittime,dischtime,intime,charttime,adm_icu_period,chart_icu_period,...,ethnicity,dob,gender,admission_age,category,description,text,class_label,note,scispacy_note
0,60039,100012,239289,EMERGENCY,2177-03-12 11:48:00,2177-03-22 14:30:00,2177-03-14 10:52:23,2177-03-12 16:40:00,1.96,1.76,...,white,2109-06-26 00:00:00,M,67.71,Radiology,CHEST (PRE-OP PA & LAT),[**2177-3-12**] 4:40 PM\n CHEST (PRE-OP PA & L...,1,Radiology\nCHEST (PRE-OP PA & LAT)\n[**2177-3-...,Radiology \n CHEST ( PRE-OP PA & LAT ) \n [ * ...
1,30078,100036,296976,EMERGENCY,2187-07-13 13:58:00,2187-07-23 15:45:00,2187-07-17 09:09:36,2187-07-16 08:34:00,3.80,1.02,...,white,2104-08-30 00:00:00,F,82.87,Radiology,CHEST (PA & LAT),[**2187-7-16**] 8:34 AM\n CHEST (PA & LAT) ...,1,Radiology\nCHEST (PA & LAT)\n[**2187-7-16**] 8...,Radiology \n CHEST ( PA & LAT ) \n [ * * 2187 ...
2,58947,100037,221136,EMERGENCY,2183-03-23 18:21:00,2183-05-09 12:00:00,2183-04-20 13:16:43,2183-03-24 06:26:00,27.79,27.29,...,white,2124-10-29 00:00:00,M,58.40,Physician,Physician Resident Progress Note,Chief Complaint:\n 24 Hour Events:\n -rece...,0,Physician \nPhysician Resident Progress Note\n...,Physician \n Physician Resident Progress Note ...
3,58947,100037,221136,EMERGENCY,2183-03-23 18:21:00,2183-05-09 12:00:00,2183-04-20 13:16:43,2183-03-23 20:47:00,27.79,27.69,...,white,2124-10-29 00:00:00,M,58.40,Physician,Physician Resident Admission Note,"Chief Complaint: Pancytopenia, ICH\n HPI:\n...",0,Physician \nPhysician Resident Admission Note\...,Physician \n Physician Resident Admission Note...
4,58947,100037,221136,EMERGENCY,2183-03-23 18:21:00,2183-05-09 12:00:00,2183-04-20 13:16:43,2183-03-24 06:26:00,27.79,27.29,...,white,2124-10-29 00:00:00,M,58.40,Physician,Physician Resident Progress Note,Chief Complaint:\n 24 Hour Events:\n -rece...,0,Physician \nPhysician Resident Progress Note\n...,Physician \n Physician Resident Progress Note ...


In [7]:
df.to_csv(path/'processed_dataset.csv', index=False)

## Stats

In [ ]:
df = pd.read_csv(path/'processed_dataset.csv')

In [ ]:
df.loc[(df['class_label'] == 0)].groupby('category').apply(lambda g: pd.Series(g['subject_id'].nunique()))

In [ ]:
df.loc[(df['class_label'] == 0)].groupby('category').size()